In [1]:
import csv
import json
import struct
from functools import reduce
import numpy as np

# trans .csv to .param

In [2]:
def trans_csv_to_param():
    with open(netName+'.param','w') as p: 
        with open(netName+'.csv','r') as f:
            f_csv = csv.reader(f)

            blobs = []
            outputcontent = ''
            row_index = 0
            for row in f_csv:   ## read rows in csv
                # normal lines <layertype   layername   inputNum outputNum inputBlob outputBlob params>
                col_index = 0
                for col in row:
                    if col_index == 0:                # layertype
                        outputcontent += '%-16s ' % (col)
                    elif col_index == 1:              # layername
                        outputcontent += '%-32s ' % (col)
                    elif col_index == 2:              # inputNum
                        inputBlobNum = int(col)
                        outputcontent += ' %s' % (col)
                    elif col_index == 3:              # outputNum
                        outputBlobNum = int(col)
                        outputcontent += ' %s' % (col)
                    elif col_index > 3 and col_index <= 3+inputBlobNum+outputBlobNum:  # inputBlob outputBlob
                        blobs.append(col)
                        outputcontent += ' %s' % (col)
                    else:                             # params
                        outputcontent += ' %s' % (col)
                    col_index += 1
                outputcontent += '\n'
                row_index += 1  ## end of read rows in csv
            outputcontent = '7767517\n'+'%s '%(row_index)+'%s\n'%(len(list(set(blobs))))+outputcontent
            p.write(outputcontent)

# process conv unit from json to csv
# save json buffer data to bin

### import json file

In [3]:
jsonpath = "/home/gx/myproj/flatbuffers/im14_without_q_att.json"
with open(jsonpath,'r') as f_json:
    model = json.load(f_json)

### append new row to csv file

In [4]:
def append_row_to_csv(appendList):
    with open(netName +'.csv','a', newline = "") as f:
        csv_writer = csv.writer(f, dialect = "excel")
        csv_writer.writerow(appendList)

### save TF-buffer data to NCNN-bin file 

In [5]:
# FROM o-h-w-i(tflite) TO o-i-h-w(ncnn)
def fix_weight_order(a,shape):
    temp = np.array(a).reshape(shape+(4,))
    print(temp.transpose(0,3,1,2,4).shape)
    return temp.transpose(0,3,1,2,4).flatten().tolist()

In [6]:
def save_imagedata(npa,filename):
    import os
    try:
        os.remove(filename)
    except:
        pass
    with open(filename,'w+') as f:
        temp = npa.flatten()

        print(npa.shape)
        index = 1
        sum = 0
        for i in temp.tolist():
            f.write(str(i)[0:8])
            f.write('\n')

In [7]:
a = [i for i in range(24000)]
b = fix_weight_order(a,(48,5,5,5))

(48, 5, 5, 5, 4)


In [8]:
#save weight & bias data from tflite-json file to ncnn-bin file
def save_data_to_bin(bufferIndex, shape=(), savefilter=True, append=True):
    bufferData = model['buffers'][bufferIndex]['data']
    flag = 'ab' if append else 'wb'
    with open(netName+'.bin',flag) as f:
        if savefilter:
            i = 0x00000000   # flag-structure: weight data need to save as float type
            f.write(struct.pack('<I',i))
            bufferData = fix_weight_order(bufferData,shape)
        for i in bufferData: # sava bufferdata, no need flag-structure
            f.write(struct.pack('B',i))
    return

### save several type of layers

In [9]:
op = model['subgraphs'][0]['operators']
tensor = model['subgraphs'][0]['tensors']
indexTfLite = 13
filterTensor = op[indexTfLite]['inputs'][1]
print(tuple(tensor[filterTensor]['shape']))

fliterShape = tuple(tensor[filterTensor]['shape'])

(48, 5, 5, 5)


In [10]:
tensor[filterTensor]['buffer']

375

In [11]:
bufferData = model['buffers'][375]['data']

In [12]:
res = b''
for i in bufferData:
    res+=(struct.pack('B',i))

In [13]:
def save_conv(name, indexTfLite, inputBlob, outputBlob):
    
    op = model['subgraphs'][0]['operators']
    tensor = model['subgraphs'][0]['tensors']
    
    filterTensor = op[indexTfLite]['inputs'][1]
    biasTensor = op[indexTfLite]['inputs'][2]

    if 'dilation_w_factor' in op[indexTfLite]['builtin_options'].keys():
        dilation_w_factor = op[indexTfLite]['builtin_options']['dilation_w_factor']
    else:
        dilation_w_factor = '1'
        
    if 'stride_w' in op[indexTfLite]['builtin_options'].keys():
        stride_w = op[indexTfLite]['builtin_options']['stride_w']
    else:
        stride_w = '1'

    fliterShape = tuple(tensor[filterTensor]['shape'])
    
    param = ['0=' + str(fliterShape[0]),                    # num_output
             '1=' + str(fliterShape[1]),                    # kernel_w   （kernel_h default）
             '2=' + str(dilation_w_factor),                 # rate
             '3=' + str(stride_w),                          # stride
             '4=-233',                                      # same
             '5=1',                                         # has bias
             '6=' + str(reduce(lambda x,y:x*y,fliterShape)) # weight
            ]
    
    append_row_to_csv(['Convolution']+[name]+['1','1']+[inputBlob,name]+param)
    save_data_to_bin(tensor[filterTensor]['buffer'],fliterShape,savefilter=True)  # save filter's buffer (weight)
    save_data_to_bin(tensor[biasTensor]['buffer'],savefilter=False)   # save bias's   buffer 
    return outputBlob

def save_slice(name, inputBlob, outputBlob1, outputBlob2):
    append_row_to_csv(['Slice']+[name]+['1','2']+[inputBlob,outputBlob1,outputBlob2]+['-23300=2,-233,-233','1=0'])
    return outputBlob1, outputBlob2

def save_interp(name, inputBlob, outputBlob):                         # ResizeNearestNeighbor 2*
    append_row_to_csv(['Interp']+[name]+['1','1']+[inputBlob,outputBlob]+['0=1','1=2.0','2=2.0']) 
    return outputBlob

def save_elu(name, inputBlob, outputBlob):
    append_row_to_csv(['ELU']+[name]+['1','1']+[inputBlob,outputBlob]+['0=1.0'])
    return outputBlob
    
def save_relu(name, inputBlob, outputBlob):
    append_row_to_csv(['ReLU']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob
    
def save_tanh(name, inputBlob, outputBlob):
    append_row_to_csv(['TanH']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob
    
def save_sigmoid(name, inputBlob, outputBlob):
    append_row_to_csv(['Sigmoid']+[name]+['1','1']+[inputBlob,outputBlob])
    return outputBlob

def save_mul(name, inputBlob1, inputBlob2, outputBlob):
    append_row_to_csv(['BinaryOp']+[name]+['2','1']+[inputBlob1,inputBlob2,outputBlob]+['0=2'])
    return outputBlob

def save_add(name, inputBlob1, inputBlob2, outputBlob):
    append_row_to_csv(['BinaryOp']+[name]+['2','1']+[inputBlob1,inputBlob2,outputBlob]+['0=0'])
    return outputBlob

def save_sub(name, inputBlob1, inputBlob2, outputBlob):
    append_row_to_csv(['BinaryOp']+[name]+['2','1']+[inputBlob1,inputBlob2,outputBlob]+['0=1'])
    return outputBlob

def save_split(name, inputBlob, outputBlob1, outputBlob2):
    append_row_to_csv(['Split']+[name]+['1','2']+[inputBlob,outputBlob1,outputBlob2])
    return outputBlob1, outputBlob2

def save_concat(name, inputBlob1, inputBlob2, outputBlob):
    append_row_to_csv(['Concat']+[name]+['2','1']+[inputBlob1,inputBlob2,outputBlob]+['0=0']) # 0 dim concat 1 h 2 w
    return outputBlob

def save_input(name, d0, d1, d2):
    append_row_to_csv(['Input']+[name]+['0','1']+[name]+['0='+str(d0)]+['1='+str(d1)]+['2='+str(d2)])
    return name

def save_memorydata(name, d0, d1, d2):
    append_row_to_csv(['MemoryData']+[name]+['0','1']+[name]+['0='+str(d0)]+['1='+str(d1)]+['2='+str(d2)])
    return name

### save 3 types of conv2d unit

In [14]:
def save_conv_unit(unitType, indexTfLite, inputBlob, deconv, stage=''):
    
    def N(name):
        return stage+'/'+name+'_'+str(indexTfLite)
    
    x = inputBlob
    
    if deconv:
        x = save_interp(('interp'), x, N('interp'))
        
    x = save_conv(N('conv2d'),indexTfLite, x, N('conv2d'))
    
    if unitType == 'TanH':
        x = save_tanh(N('tanh'),x,N('tanh'))
    else:  
        x,y = save_slice(N('slice'), x, N('conv2d')+'_A', N('conv2d')+'_B')

        if unitType == 'ELU':
            x = save_elu(N('elu'),x,N('elu'))
        if unitType == 'ReLU':
            x = save_relu(N('relu'),x,N('relu'))

        y = save_sigmoid(N('sigmoid'),y,N('sigmoid'))
        x = save_mul(N('mul'),x,y,N('mul'))   

    return x

## Build Net

In [15]:
def init_net():
    
    import os 
    try:
        os.remove(netName+'.csv')
        os.remove(netName+'.bin')
    except:
        pass
    #########   save memorydata ########
    a = [1]
    with open(netName+'.bin','wb') as f:
        for i in a:
            print(struct.pack('<f',i))
            f.write(struct.pack('<f',i))
    ####################################        

    
    x = save_input('data',512,680,3)
    y = save_input('mask',512,680,3)
    x1,x2 = save_split('b_i/split',x,'b_i_1','b_i_2')
    y1,y2 = save_split('mask/split',y,'mask_1','mask_2')
    y11,y12 = save_split('mask/split_1',y1,'mask_1_1','mask_1_2')
    m = save_memorydata('one',1,0,0)
    y11 = save_sub('sub', m, y11, 'sub')
    y111,y112 = save_split('one_sub_mask/split',y11,'sub_1','sub_2')
    i = save_input('inpaint',512,680,5)

In [16]:
init_net()  # also flash param & bin file 
stage1 = [13,20,25,30,35, 40,45,50,55,60, 65,70,76,81,87, 92,97]
index = 1
x = 'inpaint'
for i in stage1:
    unitType = 'TanH' if index == 17 else 'ELU'
    deconv = True if index in [13,15] else False
    x = save_conv_unit(unitType,i,x,deconv,'stage1')
    index += 1
    
x = save_mul('stage1/output_mask2_mul', 'mask_2', x, 'stage1/output_mask2_mul')
bm = save_mul('bi_submask', 'sub_2', 'b_i_2', 'bi_submask')
x = save_add('stage1/output_bi_add', bm, x, 'stage1/output_bi_add')
x,y = save_split('stage2',x,'stage2_A','stage2_B')

stage2_A = [101,111,121,131,141, 151,162,173,181,186]
index = 1
for i in stage2_A:
    x = save_conv_unit('ELU',i,x,False,'stage2_A')
    index += 1
    
stage2_B = [102,112,122,132,142, 152,161,170]
index = 1
for i in stage2_B:
    unitType = 'ReLU' if index == 6 else 'ELU'
    y = save_conv_unit(unitType,i,y,False,'stage2_B')
    index += 1
    
x = save_concat('concat2',x,y,'concat2')

stage2_C = [192,197,203,208,214, 219,224]
index = 1
for i in stage2_C:
    unitType = 'TanH' if index == 7 else 'ELU'
    deconv = True if index in [3,5] else False
    x = save_conv_unit(unitType,i,x,deconv,'stage2_C')
    index += 1
    

pm1 = save_mul('mul', 'b_i_1', 'sub_1', 'mul')
pm2 = save_mul('mul_1', 'mask_1_2', x, 'mul_1')
x = save_add('add', pm1, pm2, 'add')

trans_csv_to_param()

NameError: name 'netName' is not defined

In [17]:
netName = 'mynet_test'

In [ ]:
init_net()
post_net('b_i_2')
trans_csv_to_param()